In [427]:
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import requests
import spacy
import en_core_web_sm

In [297]:
df = pd.read_excel(f"C:\\Users\\Dayron Cuadros\\Documents\\Uniandes\\C3\\Visualización y storytelling\\Semana3\\Ask A Manager Salary Survey 2021 (Responses).xlsx")
df.columns = ['date', 'age', 'industry', 'job_title', 'job_context', 'annual_salary', 'estimated_benefits_amount', 'currency', 'currency_other', 'income_context', 'country', 'state', 'city', 'experience_overall', 'experience_in_field', 'education_level', 'gender', 'race']
df

,date,age,industry,job_title,job_context,annual_salary,estimated_benefits_amount,currency,currency_other,income_context,country,state,city,experience_overall,experience_in_field,education_level,gender,race
0,2021-04-27 11:02:09.743,25-34,Education (Higher Education),Research and Instruction Librarian,NaN,55000,0.0,USD,NaN,NaN,United States,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White
1,2021-04-27 11:02:21.562,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,54600,4000.0,GBP,NaN,NaN,United Kingdom,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White
2,2021-04-27 11:02:38.125,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,34000,NaN,USD,NaN,NaN,US,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White
3,2021-04-27 11:02:40.643,25-34,Nonprofits,Program Manager,NaN,62000,3000.0,USD,NaN,NaN,USA,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White
4,2021-04-27 11:02:41.793,25-34,"Accounting, Banking & Finance",Accounting Manager,NaN,60000,7000.0,USD,NaN,NaN,US,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27931,2023-01-31 18:13:07.494,35-44,Nonprofits,label editor,I work for a museum and edit the labels that g...,64000,NaN,USD,NaN,NaN,U.S.,Maryland,Baltimore,11 - 20 years,5-7 years,Master's degree,Woman,White
27932,2023-02-01 07:53:38.039,45-54,Law,Attorney,In-house counsel and other duties at a corpora...,110000,12000.0,USD,NaN,NaN,US,Virginia,Chesapeake,21 - 30 years,5-7 years,"Professional degree (MD, JD, etc.)",NaN,White
27933,2023-02-02 14:22:10.843,18-24,Computing or Tech,DevOps Engineer,NaN,90000,10000.0,USD,NaN,NaN,United States,New Jersey,Jersey CIty,2 - 4 years,2 - 4 years,Master's degree,Man,Asian or Asian American
27934,2023-02-05 23:44:56.892,18-24,Education (Higher Education),tutor,I hold several different jobs throughout the y...,27040,NaN,USD,NaN,NaN,US,Rhode Island,bristol,2 - 4 years,2 - 4 years,Some college,Woman,White


In [298]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27936 entries, 0 to 27935
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   date                       27936 non-null  datetime64[ns]
 1   age                        27936 non-null  object        
 2   industry                   27864 non-null  object        
 3   job_title                  27936 non-null  object        
 4   job_context                7231 non-null   object        
 5   annual_salary              27936 non-null  int64         
 6   estimated_benefits_amount  20685 non-null  float64       
 7   currency                   27936 non-null  object        
 8   currency_other             197 non-null    object        
 9   income_context             3034 non-null   object        
 10  country                    27936 non-null  object        
 11  state                      22956 non-null  object        
 12  city

In [299]:
sorted = df.sort_values(by=['country'], ascending=True)
pd.DataFrame(sorted.country.unique()).head(100)

,0
0,New Zealand
1,U.S.
2,US
3,United States
4,"$2,175.84/year is deducted for benefits"
...,...
95,"I earn commission on sales. If I meet quota, I..."
96,I was brought in on this salary to help with t...
97,"I work for a UAE-based organization, though I ..."
98,I work for an US based company but I'm from Ar...


In [300]:
locations = pd.read_csv('https://datahub.io/core/world-cities/r/world-cities.csv')
countries = locations.country.unique()
cities = locations.name.unique()
currencies = pd.read_csv('currency_codes.csv')

In [301]:
df['country_suggested']=df['country']

In [302]:
# Obtener los registros cuyos paises no se encuentran en la lista
df_dirty_countries = df[~df.country.isin(countries)]
df_dirty_countries.sort_values(by=['country'], ascending=True).country.unique()

array([' New Zealand', ' U.S.', ' US', ' United States',
       '$2,175.84/year is deducted for benefits',
       'ARGENTINA BUT MY ORG IS IN THAILAND', 'Africa', 'America',
       'Aotearoa New Zealand', 'Australi', 'Australia ', 'Australian ',
       'Austria, but I work remotely for a Dutch/British company',
       'Bangladesh ', 'Belgium ', 'Brasil', 'Brazil ', 'Britain ',
       'CANADA', 'CANADA ', 'California ', 'Can', 'Canad', 'Canada ',
       'Canada and USA', 'Canada, Ottawa, ontario', 'Canadw', 'Canadá',
       'Canda', 'Catalonia', 'Company in Germany. I work from Pakistan.',
       'Congo', 'Contracts', "Cote d'Ivoire", 'Croatia ', 'Csnada',
       'Currently finance', 'Czech Republic ', 'Czech republic',
       'Czechia', 'Danmark', 'Denmark ', 'ENGLAND', 'England', 'England ',
       'England, Gb', 'England, UK', 'England, UK.',
       'England, United Kingdom', 'England, United Kingdom ',
       'England/UK', 'Englang', 'FRANCE',
       'For the United States governmen

In [303]:
df['country_suggested'].mask(df.country.isin(["🇺🇸", "u.s.", "united states", "us", "america", "the united states", "the us", "u. s", "u. s.", "u.a.", "u.s", "u.s.", "u.s>", "u.sa", "ua", "u.s.a.", "usa", "u.s.a", "uxz", "united y", "united statss", "uniteed states", "usa (company is based in a us territory, i work remote)", "usa-- virgin islands", "usa, but for foreign gov't", "uniited states", "unite states", "united states", "united states of america", "unites states", "usa tomorrow", "unitef stated", "united states of american", "united stares", "united state", "united state of america", "united stated", "united stateds", "united states is america", "unitedstates", ' U.S.', ' US', ' United States', 'For the United States government, but posted overseas', 'Hartford', 'u.s.', 'uS', 'uSA', 'united States', 'united stated', 'united states', 'united states ', 'united states of america', 'us', 'usa', 'usa ', '🇺🇸 ', 'California ', 'I work for a UAE-based organization, though I am personally in the US.', 'n/a (remote from wherever I want)', 'na', 'America', 'San Francisco', 'The US', 'The United States', 'U. S ', 'U. S.', 'U. S. ', 'U.A.', 'U.S', 'U.S.', 'U.S. ', 'U.S.A', 'U.S.A ', 'U.S.A.', 'U.S.A. ', 'U.S>', 'U.SA', 'U.s.', 'U.s.a.', 'UNITED STATES', 'UNited States', 'US', 'US ', 'US govt employee overseas, country withheld', 'US of A', 'USA', 'USA ', 'USA (company is based in a US territory, I work remote)', 'USA tomorrow ', "USA, but for foreign gov't", 'Worldwide (based in US but short term trips aroudn the world)', 'USA-- Virgin Islands', 'USAB', 'USD', 'USS', 'USaa', 'United STates', 'United Sates', 'Uniited States', 'Unite States', 'United  States', 'United Sates of America', 'United Stares', 'United Stares ', 'United State', 'United State of America', 'United Statea', 'United Stated', 'United Stateds', 'United Statees', 'United States', 'United States ', 'United States (I work from home and my clients are all over the US/Canada/PR', 'United States Of America', 'United States is America', 'United States of America', 'United States of America ', 'United States of American', 'United States of American ', 'United States of Americas', 'United States of america', 'United States of america ', 'United Statesp', 'United Statss', 'United Stattes', 'United Statues', 'United Status', 'United Statws', 'United Sttes', 'United states', 'United states ', 'United states of America', 'UnitedStates', 'Virginia']), 'United States', inplace=True)
df['country_suggested'].mask(df.country.isin(['ARGENTINA BUT MY ORG IS IN THAILAND', "I work for an US based company but I'm from Argentina."]), 'Argentina', inplace=True)
df['country_suggested'].mask(df.country.isin(['Austria, but I work remotely for a Dutch/British company']), 'Austria', inplace=True)
df['country_suggested'].mask(df.country.isin(['Britain ', 'ENGLAND', 'England', 'England ', 'England, Gb', 'England, UK', 'England, UK.', 'England, United Kingdom', 'England, United Kingdom ', 'England/UK', 'Englang', 'Great Britain', 'Great Britain ', 'uk', 'united kingdom', 'Scotland', 'Scotland ', 'Scotland, UK', 'U.K', 'U.K.', 'U.K. ', 'U.K. (northern England)', 'UA', 'UAE', 'UK', 'UK ', 'UK (England)', 'UK (Northern Ireland)', 'UK for U.S. company', 'UK, but for globally fully remote company', 'UK, remote', 'Northern Ireland', 'Northern Ireland ', 'Northern Ireland, United Kingdom', 'Ireland ', 'ireland', 'Uk', 'Uk ', 'england', 'United Kindom', 'United Kingdom', 'United Kingdom ', 'United Kingdom (England)', 'United Kingdom.', 'United Kingdomk', 'United kingdom', 'United kingdom ', 'Wales (United Kingdom)', 'Wales, UK', 'Wales']), 'United Kingdom', inplace=True)
df['country_suggested'].mask(df.country.isin(['CANADA', 'CANADA ', 'Can', 'Canad', 'Canada', 'Canada ', 'Canada and USA', 'Canada, Ottawa, ontario', 'Canadw', 'Canadá', 'Canda', 'Csnada', 'I am located in Canada but I work for a company in the US', 'canada']), 'Canada', inplace=True)
df['country_suggested'].mask(df.country.isin(['Congo']), 'Republic of the Congo', inplace=True)
df['country_suggested'].mask(df.country.isin(['Mainland China']), 'China', inplace=True)
df['country_suggested'].mask(df.country.isin(['Company in Germany. I work from Pakistan.']), 'Pakistan', inplace=True)
df['country_suggested'].mask(df.country.isin(['$2,175.84/year is deducted for benefits', 'Global', 'International', 'International ', 'Catalonia', 'Contracts', 'Currently finance', "I earn commission on sales. If I meet quota, I'm guaranteed another 16k min. Last year i earned an additional 27k. It's not uncommon for people in my space to earn 100k+ after commission. ", 'I was brought in on this salary to help with the EHR and very quickly was promoted to current position but compensation was not altered. ', 'I.S.', 'IS', 'ISA', 'Isle of Man', 'Jersey, Channel islands', 'Latvia', 'Liechtenstein', 'europe', 'Africa', 'Policy', 'Remote', 'Remote (philippines)', 'Y', 'bonus based on meeting yearly goals set w/ my supervisor', 'UXZ', "We don't get raises, we get quarterly bonuses, but they periodically asses income in the area you work, so I got a raise because a 3rd party assessment showed I was paid too little for the area we were located"]), 'Other', inplace=True)
df['country_suggested'].mask(df.country.isin(['From Romania, but for an US based company',]), 'Romania', inplace=True)
df['country_suggested'].mask(df.country.isin(['Italy (South)']), 'Italy', inplace=True)
df['country_suggested'].mask(df.country.isin(['Japan, US Gov position', 'japan', 'Japan ']), 'Japan', inplace=True)
df['country_suggested'].mask(df.country.isin(['NL', 'NZ', 'Nederland', 'netherlands', 'the Netherlands', 'the netherlands', 'Netherlands ', 'The Netherlands', 'The Netherlands ', 'The netherlands']), 'Netherlands', inplace=True)
df['country_suggested'].mask(df.country.isin([' New Zealand', 'Aotearoa New Zealand', 'From New Zealand but on projects across APAC', 'new zealand', 'New Zealand ', 'New Zealand Aotearoa', 'New zealand']), 'New Zealand', inplace=True)
df['country_suggested'].mask(df.country.isin(['United States- Puerto Rico']), 'Puerto Rico', inplace=True)


In [304]:
df.sort_values(by=['country_suggested'], ascending=True).country_suggested.unique()

array(['Afghanistan', 'Argentina', 'Australi', 'Australia', 'Australia ',
       'Australian ', 'Austria', 'Bangladesh', 'Bangladesh ', 'Belgium',
       'Belgium ', 'Bermuda', 'Bosnia and Herzegovina', 'Brasil',
       'Brazil', 'Brazil ', 'Bulgaria', 'Cambodia', 'Canada',
       'Cayman Islands', 'Chile', 'China', 'Colombia', 'Costa Rica',
       "Cote d'Ivoire", 'Croatia ', 'Cuba', 'Cyprus', 'Czech Republic',
       'Czech Republic ', 'Czech republic', 'Czechia', 'Danmark',
       'Denmark', 'Denmark ', 'Ecuador', 'Eritrea', 'Estonia', 'FRANCE',
       'Finland', 'France', 'France ', 'Germany', 'Germany ', 'Ghana',
       'Greece', 'Hong Kong', 'Hong Kong ', 'Hungary', 'INDIA', 'India',
       'Indonesia', 'Ireland', 'Israel', 'Italy', 'Jamaica', 'Japan',
       'Jordan', 'Kenya', 'Kuwait', 'Lithuania', 'Luxembourg',
       'Luxemburg', 'Malaysia', 'Malaysia ', 'Malta', 'Mexico', 'Mexico ',
       'Morocco', 'México', 'NIGERIA', 'Netherlands', 'New Zealand',
       'Nigeria', 'Norwa

In [305]:
df[~df['country_suggested'].notnull()].sort_values(by=['country'], ascending=True).country.unique()

array([], dtype=object)

In [340]:
def suggest_standard_text(reference, dirty_name):
    return process.extractOne(dirty_name, reference, scorer=fuzz.token_sort_ratio)[0]

df['country_standardized'] = df['country_suggested']
df['country_standardized'] = [c if c=='Other' else suggest_standard_text(countries, c) for c in df.country_suggested]

In [307]:
df.sort_values(by=['country_standardized'], ascending=True).country_standardized.unique()

array(['Afghanistan', 'Argentina', 'Australia', 'Austria', 'Bahamas',
       'Bangladesh', 'Belgium', 'Bermuda', 'Bosnia and Herzegovina',
       'Brazil', 'Bulgaria', 'Cambodia', 'Canada', 'Cayman Islands',
       'Chile', 'China', 'Colombia', 'Costa Rica', 'Croatia', 'Cuba',
       'Cyprus', 'Czech Republic', 'Denmark', 'Ecuador', 'Eritrea',
       'Estonia', 'Finland', 'France', 'Germany', 'Ghana', 'Greece',
       'Hong Kong', 'Hungary', 'India', 'Indonesia', 'Ireland', 'Israel',
       'Italy', 'Ivory Coast', 'Jamaica', 'Japan', 'Jordan', 'Kenya',
       'Kuwait', 'Lithuania', 'Luxembourg', 'Malaysia', 'Malta', 'Mexico',
       'Morocco', 'Netherlands', 'New Zealand', 'Nigeria', 'Norway',
       'Other', 'Pakistan', 'Panama', 'Philippines', 'Poland', 'Portugal',
       'Puerto Rico', 'Qatar', 'Republic of the Congo', 'Romania',
       'Russia', 'Rwanda', 'Saudi Arabia', 'Serbia', 'Sierra Leone',
       'Singapore', 'Slovakia', 'Slovenia', 'Somalia', 'South Africa',
       'South K

In [308]:
# Revisar las ciudades que no hacen match
cities_lowercase = np.char.lower(np.array([x.lower() if isinstance(x, str) else x for x in cities]))
cities_lowercase = np.unique(cities_lowercase)
cities_lowercase

df['city_standardized'] = [c.title() if c in cities_lowercase else 'Other' for c in df.city.str.lower()]
df['city_standardized'].unique()

C:\Users\Dayron Cuadros\AppData\Local\Temp\ipykernel_28324\1392525183.py:6: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



array(['Boston', 'Cambridge', 'Chattanooga', ..., 'Port Harcourt',
       'Newbury', 'Colorado'], dtype=object)

In [309]:
df.currency.value_counts()

USD        23279
CAD         1664
GBP         1585
EUR          637
AUD/NZD      500
Other        156
CHF           37
SEK           37
JPY           23
ZAR           14
HKD            4
Name: currency, dtype: int64

In [310]:
# Dejar un solo codigo para los casos AUD/NZD
df['currency_precleaned'] = [c if c in currencies else 'AUD' if c == 'AUD/NZD' else c for c in df.currency]
df['currency_precleaned'].value_counts()

USD      23279
CAD       1664
GBP       1585
EUR        637
AUD        500
Other      156
CHF         37
SEK         37
JPY         23
ZAR         14
HKD          4
Name: currency_precleaned, dtype: int64

In [311]:
# Dado que los de otras monedas corresponden al 0.5%, se opta por excluirlos
df = df[~(df.currency_precleaned == 'Other')]

#Asignar cero a las columnas numéricas con valores nulos
df[['annual_salary', 'estimated_benefits_amount']].fillna(value=0)
df['estimated_benefits_amount'].replace(np.nan, 0, inplace=True)

C:\Users\Dayron Cuadros\AppData\Local\Temp\ipykernel_28324\4077298348.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [312]:
def get_exchange_rates(reference):
    url = f'https://v6.exchangerate-api.com/v6/{apiKey}/latest/{reference}'
    response = requests.get(url)
    return response.json()

def get_exchange_rate(exchange_rates, currency):
    return exchange_rates['conversion_rates'][currency]

exchange_rates = get_exchange_rates('COP')
get_exchange_rate(exchange_rates, 'USD')

0.00021006

In [314]:
df['annual_salary_COP'] = df.apply(lambda i: i.annual_salary / get_exchange_rate(exchange_rates, i.currency_precleaned), axis=1)
df['estimated_benefits_amount_COP'] = df.apply(lambda i: i.estimated_benefits_amount / get_exchange_rate(exchange_rates, i.currency_precleaned), axis=1)

C:\Users\Dayron Cuadros\AppData\Local\Temp\ipykernel_28324\1763369840.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Dayron Cuadros\AppData\Local\Temp\ipykernel_28324\1763369840.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [315]:
df['annual_salary_COP'].describe()

count    2.778000e+04
mean     4.338191e+08
std      2.929996e+09
min      0.000000e+00
25%      2.523089e+08
50%      3.522803e+08
75%      5.014353e+08
max      4.855755e+11
Name: annual_salary_COP, dtype: float64

In [316]:
df['estimated_benefits_amount_COP'].describe()

count    2.778000e+04
mean     4.170708e+07
std      1.819624e+08
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      2.380272e+07
max      7.140817e+09
Name: estimated_benefits_amount_COP, dtype: float64

In [320]:
# Retirar los que tienen salarios sospechosamente muy bajos o muy altos
df = df.query('annual_salary_COP >= 1000000 & annual_salary_COP <= 5000000000')[df.columns]

# Retirar los que tienen beneficios sospechosamente muy altos
df = df.query('estimated_benefits_amount_COP <= 100000000')[df.columns]

# Crear nueva columna sumando el salario y los beneficios
df['total_salary_income'] = df['annual_salary_COP'] + df['estimated_benefits_amount_COP']

In [321]:
import plotly.express as px
fig = px.histogram(df, x="annual_salary_COP", nbins=1000)
fig.show()

In [482]:
# Identificar los cargos únicos
jobs_descriptions = df["job_title"].str.cat(sep=" ").lower()
unique_descriptions = list(set(jobs_descriptions.split()))
unique_descriptions = " ".join(unique_descriptions)
unique_descriptions

# Identificar las palabras base que identifican la profesión
nlp = en_core_web_sm.load()
suffixes = ['er', 'ist', 'ant', 'ian', 'man', 'woman', 'or', 'ney']

doc = nlp(unique_descriptions)
jobs_base = []

for token in doc:
    # if token.pos_ == "ADJ":# and [s for s in suffixes if token.text.endswith(s)]:
    if (token.pos_ == "ADJ" or token.pos_ == "NOUN") and [s for s in suffixes if token.text.endswith(s)]:
        jobs_base.append(token.text)

jobs_base = set(jobs_base)

# jobs = pd.DataFrame(set(jobs_base))
# jobs.columns=['title']
# jobs

# Asignar la palabra clave del cargo
def get_suggested_job(row):
    for job in jobs_base:
        if job in row['job_title'].lower():
            return job.title()
    return 'Other'

# Aplicar la función a cada fila del DataFrame
df['job_title_standardized'] = None
df['job_title_standardized'] = df.apply(get_suggested_job, axis=1)
df


,date,age,industry,job_title,job_context,annual_salary,estimated_benefits_amount,currency,currency_other,income_context,...,gender,race,country_suggested,country_standardized,city_standardized,currency_precleaned,annual_salary_COP,estimated_benefits_amount_COP,total_salary_income,job_title_standardized
0,2021-04-27 11:02:09.743,25-34,Education (Higher Education),Research and Instruction Librarian,NaN,55000,0.0,USD,NaN,NaN,...,Woman,White,United States,United States,Boston,USD,2.618300e+08,0.000000e+00,2.618300e+08,Librarian
1,2021-04-27 11:02:21.562,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,54600,4000.0,GBP,NaN,NaN,...,Non-binary,White,United Kingdom,United Kingdom,Cambridge,GBP,3.154793e+08,2.311204e+07,3.385913e+08,Manager
2,2021-04-27 11:02:38.125,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,34000,0.0,USD,NaN,NaN,...,Woman,White,United States,United States,Chattanooga,USD,1.618585e+08,0.000000e+00,1.618585e+08,Specialist
3,2021-04-27 11:02:40.643,25-34,Nonprofits,Program Manager,NaN,62000,3000.0,USD,NaN,NaN,...,Woman,White,United States,United States,Milwaukee,USD,2.951538e+08,1.428163e+07,3.094354e+08,Manager
4,2021-04-27 11:02:41.793,25-34,"Accounting, Banking & Finance",Accounting Manager,NaN,60000,7000.0,USD,NaN,NaN,...,Woman,White,United States,United States,Greenville,USD,2.856327e+08,3.332381e+07,3.189565e+08,Manager
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27931,2023-01-31 18:13:07.494,35-44,Nonprofits,label editor,I work for a museum and edit the labels that g...,64000,0.0,USD,NaN,NaN,...,Woman,White,United States,United States,Baltimore,USD,3.046749e+08,0.000000e+00,3.046749e+08,Editor
27932,2023-02-01 07:53:38.039,45-54,Law,Attorney,In-house counsel and other duties at a corpora...,110000,12000.0,USD,NaN,NaN,...,NaN,White,United States,United States,Other,USD,5.236599e+08,5.712654e+07,5.807864e+08,Attorney
27933,2023-02-02 14:22:10.843,18-24,Computing or Tech,DevOps Engineer,NaN,90000,10000.0,USD,NaN,NaN,...,Man,Asian or Asian American,United States,United States,Jersey City,USD,4.284490e+08,4.760545e+07,4.760545e+08,Engineer
27934,2023-02-05 23:44:56.892,18-24,Education (Higher Education),tutor,I hold several different jobs throughout the y...,27040,0.0,USD,NaN,NaN,...,Woman,White,United States,United States,Bristol,USD,1.287251e+08,0.000000e+00,1.287251e+08,Tutor


In [483]:
df.job_title_standardized.unique()

array(['Librarian', 'Manager', 'Specialist', 'Assistant', 'Other',
       'Senior', 'Attorney', 'Coordinator', 'Designer', 'Programmer',
       'Planner', 'Researcher', 'Director', 'Economist', 'Engineer',
       'Editor', 'Advisor', 'Reporter', 'Receptionist', 'Writer',
       'Developer', 'Teacher', 'Water', 'Officer', 'Supervisor',
       'Educator', 'Scientist', 'Cataloger', 'Webmaster', 'Administrator',
       'Archivist', 'Ordinator', 'Customer', 'Rofessor', 'Acountant',
       'Donor', 'Sector', 'Career', 'Estimator', 'Technologist', 'User',
       'Technician', 'Consultant', 'Strategist', 'Provider', 'Partner',
       'Assitant', 'Chemist', 'Investigator', 'Adviser', 'Accountant',
       'Keeper', 'Recruiter', 'Grant', 'Worker', 'Counselor', 'Leader',
       'Pharmacist', 'Controller', 'Pathologist', 'Transcriptionist',
       'Auditor', 'Historian', 'Therapist', 'Doctor', 'Processor',
       'Producer', 'Member', 'Instructor', 'Buyer', 'Psychometrist',
       'Lecturer', 'Labo

In [485]:
df.to_excel(f'AskAManagerSalaries2021_precleaned.xlsx')